## Preprocessing

In [54]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()

In [28]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

#Setting our threshold for what to call "other".
other_threshold = 500

#Defining our applications to replace, as those with counts less than our "other" threshold.
apptypes_to_replace = application_type_counts[application_type_counts < other_threshold].index
#Having them correctly labeled in the df.
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(apptypes_to_replace, 'Other')

print(application_type_counts)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [30]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Replace in dataframe
for app in apptypes_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [32]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()

#I chose 1000 as our "other" threshold, as it seems to be where numbers get quite small relatively. 
other_class_threshold = 1000 

classifications_to_replace = classification_counts[classification_counts < other_class_threshold].index
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, 'Other')

print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: count, dtype: int64


In [33]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_over1 = classification_counts[classification_counts > 1]
classification_counts_over1
#Appears to be the same count.

CLASSIFICATION
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: count, dtype: int64

In [34]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 2000
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: count, dtype: int64

In [35]:
# Convert categorical data to numeric with `pd.get_dummies`
df_numeric = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])

In [36]:
# Split our preprocessed data into our features and target arrays
X = df_numeric.drop(columns=['IS_SUCCESSFUL'])
y = df_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=38)


In [37]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [59]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#nn = tf.keras.models.Sequential()
#Setting our features correctly, re the number of columns we have, features.
#input_features = len(X.columns)

# First hidden layer
#Using relu, rectified linear unit as our activation function
#for the first layer we specify the unput features, following layers simply infer based on previous output
#nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=input_features))

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
#Only 1 unit here as output is either 0 or 1. Binary.
#Sigmoid as our activation function allows us to achieve the above.
#nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#Model accuracy not high enough to meet requirements, so trying a Dropout and an early stopping
#in an attempt to avoid overfitting. However, it did not work well enough, and this model still does not reach
#the accuracy requirements.
nn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu', input_dim=input_features),
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Check the structure of the model
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 128)            │         5,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,825 (54.00 KB)

 Trainable params: 13,825 (54.00 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
# Compile the model
#The loss measures how well our predictions match target, since we have a binary 
#classifcation problem we use binary_crossentropy.
#Adam optimizes for us.
#We include accuracy in our metrics to see how well the model is matching either a 0 or a 1 prediction output.
nn.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [57]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=64, verbose=1, 
                 validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

Epoch 1/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7111 - loss: 0.5902 - val_accuracy: 0.7208 - val_loss: 0.5689
Epoch 2/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7285 - loss: 0.5667 - val_accuracy: 0.7184 - val_loss: 0.5639
Epoch 3/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7323 - loss: 0.5571 - val_accuracy: 0.7195 - val_loss: 0.5620
Epoch 4/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7266 - loss: 0.5603 - val_accuracy: 0.7195 - val_loss: 0.5630
Epoch 5/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7277 - loss: 0.5562 - val_accuracy: 0.7203 - val_loss: 0.5610
Epoch 6/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - accuracy: 0.7266 - loss: 0.5579 - val_accuracy: 0.7233 - val_loss: 0.5596
Epoch 7/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7317 - loss: 0.5548 - val_accuracy: 0.7226 - val_loss: 0.5592
Epoch 8/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.7338 - loss: 0.5516 - val_

In [58]:
# Evaluate the model using the test data
nn_model_loss, nn_model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"NN Model Loss: {nn_model_loss}, NN Model Accuracy: {nn_model_accuracy}")

215/215 - 0s - 596us/step - accuracy: 0.7230 - loss: 0.5563
NN Model Loss: 0.5562537908554077, NN Model Accuracy: 0.7230320572853088


In [60]:
# Export our model to HDF5 file
nn.save('nn_model_module_21.h5')